In [14]:
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import shutil
import time
import csv
import os

## Funzione per trasformare i dati in CSV
Lo script prende in input un file .txt con il parametro 'rb' quindi in lettura in formato bytecode che vuol dire che dovrà essere decodificato.
Per prima cosa si scorre fino alla riga che inizia con 'NR' perché i dati iniziano da li, quando la trovo imposto flag_data a 1 che serve a dirmi che da li in poi dovrò salvare le righe.
Cerco la prima riga che inizia con '0' e la salto settando firsr_row a 1.
Quando sia data_flag che first_row sono a 1 allora inizio a salvare le righe.

Per salvare le righe:
* prima le decodifico con .decode()
* poi le divido con .split() che se usato senza parametri splitta la stringa in base agli spazi
* unisco i valori delle ultime due colonne perché fanno entrambi riferimento alla data
* elimino l'ultima riga perché l'ho unita alla penultima
* faccio un append della riga


In [22]:
def txt_to_csv(filename, filedest, tab_type):
    
    
    if os.path.isfile(filedest + '.csv'):
        os.remove(filedest + '.csv')
        
    file1 = open(filename + tab_type, "rb")
    file_csv = open(filedest + '.csv', 'w+', newline ='')

    count = 0
    flag_data = 0

    with file_csv:
        for line in file1:

            if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
                pass
            else:
                line = line.decode() # altrimenti decodifico la riga

                # cerco la riga con i nomi della tabella
                if line.startswith('NR'):
                    flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                    table_names = line.split()     # divido i nomi delle colonne      
                    table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                    #for item in table_names: print(item)
                    write = csv.writer(file_csv)  
                    write.writerow(table_names)
                    #print(table_names)

                # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
                elif flag_data == 1 and line.startswith('0'):
                    pass

                # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
                # inizio a salvare i dati
                elif flag_data == 1:
                    row = line.split()

                    if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                        row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                        del row[-1] # elimino la colonna con i giorni
                        write = csv.writer(file_csv)
                        write.writerow(row)
                        count = count+1      # aggiorno il contatore delle righe

    #print(str(count) + ' righe salvate.')

#txt_to_csv(filename = 'table/DiagnosisTabl',
#           filedest = 'table/csv/DiagnosisTabl')

Per convertire tutti i file da txt a csv mi basta iterare all'interno della cartella e passare ogni file alla funzione txt_to_csv che prende in input il nome del file da convertire e il percorso di destinazione del file.

In [23]:
path = 'table_new/'
path_dest = path + 'csv/'
tab_type = '.tab'

# controllo se la cartella 'csv' è presente. Se c'è la elimino e la ricreo, se non c'è la creo.
if os.path.isdir(path_dest):
    shutil.rmtree(path_dest)
    os.mkdir(path_dest)
else:
    os.mkdir(path_dest)

for filename in os.listdir(path):
    if filename.endswith(tab_type): 
        print(filename[:-4])
        txt_to_csv(filename = path + filename[:-4],
                   filedest = path_dest + filename[:-4],
                   tab_type = tab_type)

Long_term_temp
DomoClima_temp
Diag_temp
hour_air_consumption


## Funzione per caricare il CSV in un DataFrame
La funzione prende in input il nome del file .csv da caricare, converte la colonna DataTime in datetime e la imposta come indice del dataframe. Controlla se il dataframe è ordinato e se non lo è lo riordina. Elimina la colonna NR. Restituisce il dataframe.

In [8]:
def load_data(file):
    #column_subset = ['t_Mach',   't_HydrTa', 't_TTlubr', 't_TT_ab1', 
                     #'t_TT_ab2', 't_HdStUp', 't_HdStLw', 't_Sfrnt1', 
                     #'t_Sfrnt2', 't_Srear',  't_Smotor', 'DataTime']

    df = pd.read_csv (file,
                      #usecols=column_subset,
                      index_col=False)

    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%Y')
    
    df.set_index('DataTime', drop = True, inplace=True)
    
    if not df.index.is_monotonic:
        df.sort_index(inplace = True)
    
    df.drop('NR', axis=1, inplace=True)
    
    return df

## Processo per caricare tutti i file dello stesso tipo e salvarli come unico csv

In [ ]:
def create_single_csv(path_folder, tab_name):
    data_list=[]

    for filename in os.listdir(path_folder):
        if tab_name in filename: 
            data = load_data(path_folder + 'csv/' + filename[:-4] + '.csv')
            data_list.append(data)

    data_all = pd.concat(data_list)
    data_all.sort_index(inplace = True)
    data_all.to_csv(path_folder + '/csv/' + tab_name + '_all.csv')


create_single_csv('table/', 'DX')

 * Faccio una lista con tutti i nomi dei file ma senza il numero progressivo alla fine
 * Per ogni nome in lista cerco le tabelle con quel nome e le riunisco creando un unico file csv

In [ ]:
import os
from tqdm import tqdm
import shutil

# controllo se la cartella 'csv' è presente. Se c'è la elimino e la ricreo, se non c'è la creo.
if os.path.isdir('table/csv'):
    shutil.rmtree('table/csv')
    os.mkdir('table/csv')
else:
    os.mkdir('table/csv')

#per ogni file .txt creo il corrispettivo in .csv nella cartella table/csv
table_list = []
pbar = tqdm(os.listdir('table/'), desc='converto .txt in .csv\t')
for filename in pbar:
    if filename.endswith(".txt"):
        #t.set_description("Bar desc (file %i)" % i)
        print(filename[:-4])
        txt_to_csv(filename = 'table/' + filename[:-4],
                   filedest = 'table/csv/' + filename[:-4])
        
        # creo una lista con i nomi delle tabelle da unire
        if not filename[-5:-4].isdigit():
            table_list.append(filename[:-4])

# unisco i csv
pbar = tqdm(table_list, desc='unisco le tabelle\t')
for table in pbar:
    create_single_csv('table/', table)

# elimino i file originali e tengo solo i .csv uniti
pbar = tqdm(os.listdir('table/csv'), desc='elimino tab divise\t')
for filename in pbar:
    if not filename.endswith('_all.csv'):
        os.remove('table/csv/'+filename)
        
print(table_list)

In [ ]:
create_single_csv('table/', 'TabLogFMS_DX')
df = pd.read_csv ('table/csv/TabLogFMS_DX_all.csv', index_col='DataTime')
df.index = pd.to_datetime(df.index)
df.plot()

## Caricamento dati in modo manuale

In [10]:
def CARICA_TABELLE_BRUTTO():

    #--- carico tutti i file TabLogFMS_DX
    dflist=[]
    for file in os.listdir('table/csv'):
        if 'TabLogFMS' in file and '_DX' in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/TabLogFMS_DX_all.csv')    

    #--- carico tutti i file TabLogFMS_SX
    dflist=[]
    for file in os.listdir('table/csv'):
        if 'TabLogFMS' in file and '_SX' in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/TabLogFMS_SX_all.csv')

    #--- carico tutti i file TabLogPostTHS
    dflist=[]
    filename = 'TabLogPostTHS'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file DiagnosisTabl
    dflist=[]
    filename = 'DiagnosisTabl'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file TabLogAntTHS
    dflist=[]
    filename = 'TabLogAntTHS'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file DiagnTableBasaments
    dflist=[]
    filename = 'DiagnTableBasaments'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file LONG_TERM_TEMP
    dflist=[]
    filename = 'LONG_TERM_TEMP'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

# elimino i file originali e tengo solo i .csv uniti che carico in un dizionario di dataframe
dataDict={}
for filename in os.listdir('table/csv'):
    if not filename.endswith('_all.csv'):
        os.remove('table/csv/'+filename)
    elif filename.endswith('_all.csv'):
        print(filename)
        df = pd.read_csv ('table/csv/'+filename, index_col='DataTime')
        df.index = pd.to_datetime(df.index)
        df = df.resample('5T').mean()
        dataDict[filename[:-4]] = df


TabLogAntTHS_all.csv
DiagnTableBasaments_all.csv
TabLogFMS_SX_all.csv
LONG_TERM_TEMP_all.csv
TabLogPostTHS_all.csv
DiagnosisTabl_all.csv
TabLogFMS_DX_all.csv


In [ ]:
dataDict={}
for filename in os.listdir('table/csv'):
    print(filename)
    df = pd.read_csv ('table/csv/'+filename, index_col='DataTime')
    dataDict[filename[:-4]] = df


dataDict['TabLogAntTHS_all'].head()

In [21]:
#--- TABLE NEW

tabname = 'Diag_temp'
path = 'table_new/csv/'
dflist=[]
for file in os.listdir(path):
    if file.endswith('.csv') and tabname in file:
        print(file)
        df = load_data(path + file)
        dflist.append(df)
        print(df.head(50)['t_TTlubr'])
        
data_all = pd.concat(dflist)
data_all.sort_index(inplace = True)
data_all.to_csv(path + filename + '_all.csv')

data_all.info()

Diag_temp2.csv
DataTime
2021-02-25 10:49:01    21.5
Name: t_TTlubr, dtype: float64
Diag_temp4.csv
DataTime
2021-02-25 10:49:01    21.5
Name: t_TTlubr, dtype: float64
Diag_temp.csv
DataTime
2021-02-25 10:49:01    21.5
Name: t_TTlubr, dtype: float64
Diag_temp3.csv
DataTime
2021-02-25 10:49:01    21.5
Name: t_TTlubr, dtype: float64
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4 entries, 2021-02-25 10:49:01 to 2021-02-25 10:49:01
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_TTlubr  4 non-null      float64
 1   t_TT_ab1  4 non-null      float64
 2   t_TT_ab2  4 non-null      float64
 3   t_Sfrnt1  4 non-null      float64
 4   t_Sfrnt2  4 non-null      float64
 5   t_Srear   4 non-null      int64  
 6   t_Smotor  4 non-null      int64  
 7   OpMode    4 non-null      object 
 8   S_Speed   4 non-null      int64  
 9   S_Load    4 non-null      int64  
 10  tCo_H_X1  4 non-null      int64  
 11  tCo_H_Y1  4 non-n